In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.f_available_stock_dc 
    TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH 
f_sto_pending AS (
    SELECT    
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
        dc_id,
        product_id,
        spr_dc,
        SUM(po_qty) AS po_qty,
        SUM(gi_qty) AS gi_qty
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sto_pending_21d
    WHERE IFNULL(gi_qty, 0) = 0 AND spr_dc = 0
    GROUP BY 1,2,3,4
), 
list_spr_dc AS (
    SELECT DISTINCT dc_id 
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive 
    WHERE 
        supra = 'YES'
        AND replenishment_mode = 'BS'
        AND dc_id IN ('1291', '1312', '1358', '1362')
), 
wcm_dc AS (
    SELECT    
        COALESCE(a.store_id, b.dc_id, c.store_id) AS dc_id,
        COALESCE(a.product_id, b.product_id, c.product_id) AS product_id,
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
        IFNULL(a.closing_stock_quantity, 0) AS stock_sloc2000_qty,
        IFNULL(c.closing_stock_quantity, 0) AS stock_sloc3005_qty,
        IFNULL(b.po_qty, 0) AS sto_pending_qty,
        IFNULL(a.closing_stock_quantity, 0) - IFNULL(b.po_qty, 0) AS avaiable_stock_qty,
        IFNULL(a.closing_stock_quantity, 0) AS physical_stock_qty,
        CAST(NULL AS DOUBLE) AS lost_qty,
        CAST(NULL AS DOUBLE) AS damage_qty,
        CAST(NULL AS DOUBLE) AS out_of_date_qty,
        CAST(NULL AS DOUBLE) AS nearing_expiration_date_qty,
        CAST(NULL AS DOUBLE) AS return_to_vendor_qty,
        0 AS spr_dc
    FROM (
        SELECT * 
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_2000 
        WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
    ) a
    FULL OUTER JOIN f_sto_pending b 
        ON a.store_id = b.dc_id AND a.product_id = b.product_id
    FULL OUTER JOIN (
        SELECT * 
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sloc_3005 
        WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1
    ) c
        ON IFNULL(a.store_id, b.dc_id) = c.store_id 
        AND IFNULL(a.product_id, b.product_id) = c.product_id
    WHERE COALESCE(a.store_id, b.dc_id, c.store_id) IN (
        SELECT store_id 
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store 
        WHERE business_unit = '1100'
    )
    AND COALESCE(a.store_id, b.dc_id, c.store_id) NOT IN (
        SELECT dc_id FROM list_spr_dc
    )
), 
spr_dc AS (
    SELECT    
        dc_id,
        product_id,
        DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - 1 AS calday,
        CAST(NULL AS DOUBLE) AS stock_sloc2000_qty,
        CAST(NULL AS DOUBLE) AS stock_sloc3005_qty,
        CAST(NULL AS DOUBLE) AS sto_pending_qty,
        rkt_available_qty AS avaiable_stock_qty,
        sft_available_qty AS physical_stock_qty,
        sft_lost_qty AS lost_qty,
        sft_damage_qty AS damage_qty,
        sft_out_of_date_qty AS out_of_date_qty,
        sft_nearing_expiration_date_qty AS nearing_expiration_date_qty,
        sft_return_to_vendor_qty AS return_to_vendor_qty,
        1 AS spr_dc
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sft 
    WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        AND dc_id IN (
            SELECT dc_id FROM list_spr_dc
        )
), 
main AS (
    SELECT * FROM wcm_dc
    UNION ALL 
    SELECT * FROM spr_dc
)

SELECT 
    FARM_FINGERPRINT(CONCAT(
        IFNULL(CAST(dc_id AS STRING), ''), 
        IFNULL(CAST(product_id AS STRING), ''),
        IFNULL(CAST(calday AS STRING), '')
    )) AS hash_id,
    *,
    CAST(NULL AS STRING) AS note
FROM main
""")
